The goal of this notebook is to compare two name-matching methods: `Standard Name Matching` and `Grouped Name Matching`.

The notebook is structured as follows:

- Preprocessing

- Summary Statistics: Overview of labeling outcomes across the dataset.

- Labeling Validation: Evaluating how well the methods categorize name pairs.

- Disagreement Analysis: Comparing the methods’ choices for best-match.

- Conclusion

- Future Improvements

- Key Limitations



# 1. Imports

In [1]:
#necessary libraries
from pathlib import Path
import pandas as pd  
import numpy as np  
import warnings  
from unidecode import unidecode
import re  
import matplotlib.pyplot as plt
import seaborn as sns
import time
from rapidfuzz import process, fuzz
import random
import scipy.stats as st
import math
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
 
#commands for better output readability 
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)  
warnings.filterwarnings("ignore", category=UserWarning, module='pandas')  
pd.set_option('display.max_colwidth', None)

In [2]:
#paths
project_dir=Path.cwd().parent.parent
processed_dir=project_dir/'data'/'processed'

matched_standard_file=processed_dir/'matched_standard.pkl'
matched_grouped_file=processed_dir/'matched_grouped.pkl'

df_standard=pd.read_pickle(matched_standard_file)  
df_grouped=pd.read_pickle(matched_grouped_file)  

# 2. Preprocessing

In [5]:
df_standard.head()

,UK ID,UK Sanction Programme,UK Name,EU Name Match,EU ID,Score,String Length,Label
0,6894,ISIL (Da'esh) and Al-Qaida,Abdul Rahman Mohamad Iqbal,Mohamad Iqbal Abdul Rahman,1004,100,26,match
1,6895,Afghanistan,Abdul Hai Hazem Abdul Qader,Abdul Hai Hazem Abdul Qader,505,100,27,match
2,6897,ISIL (Da'esh) and Al-Qaida,Abdul Man Am Saiyid,Abdul Man Am Saiyid,514,100,19,match
3,6899,ISIL (Da'esh) and Al-Qaida,Tharwat Salah Shihata Ali,Tharwat Salah Shihata Ali,796,100,25,match
4,6901,ISIL (Da'esh) and Al-Qaida,Majeed Abdul Chaudhry,Majeed Chaudhry Abdul,641,100,21,match


In [6]:
df_grouped.head()

,UK ID,UK Sanction Programme,UK Name,UK Letters,Candidate EU IDs,Candidate Count,Name Overlap,EU Name Match,EU ID,Multi Score,Coverage Ratio,Length Adjusted Scores,Weighted Score,Raw Scores,Label
0,6894,ISIL (Da'esh) and Al-Qaida,"{Fihiruddin, Abu, Muqti, A, Mohamad, Rahman, Jibril, Abdul, Abdurrahman, Iqbal, Fikiruddin}","{F, A, R, J, I, M}","[630, 643, 1004, 3140, 4686, 5240, 5262, 5271, 5623, 6133, 6211, 6478, 6494, 6830, 6974, 7250, 113355, 115714, 117974, 123615, 125562, 126101, 127538, 129864, 130225, 133935, 134828, 135060, 136530, 136975, 138176, 145803, 146560, 147032, 150914, 159126, 162479, 162975, 165652, 166799, 167049, 167477, 171171, 172374, 172394]",45,"[Jibril, Abdurrahman, Abu, A, Rahman, Fihiruddin, Mohamad, Iqbal, Muqti]","{Fihiruddin, Abu, Muqti, A, Mohamad, Rahman, Jibril, Abdul, Abdurrahman, Iqbal, Fikiruddin}",1004,97,1.0,"[100.0, 100, 87.0, 87.0, 100.0, 100, 100.0, 95.0, 95.0]",97.00,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]",match
1,6895,Afghanistan,"{Hai, Qader, Hazem, Abdul}","{Q, H, A}","[20, 505, 590, 591, 595, 603, 651, 661, 706, 709, 829, 842, 2193, 5270, 5416, 6078, 6093, 6095, 6113, 6130, 6207, 6224, 6240, 6312, 6583, 6616, 6694, 6873, 6887, 6898, 7085, 7146, 7368, 7386, 7447, 7501, 7513, 7586, 105305, 106138, 107163, 109900, 110138, 113244, 115145, 119228, 124306, 126530, 126538, 126554, 127417, 128186, 141403, 141872, 144861, 145430, 145486, 145521, 145691, 146428, 148293, 149178, 149390, 150680, 151879, 151903, 152704, 153011, 154125, 165849, 166346, 170630]",72,"[Abdul, Qader, Hazem, Hai]","{Hai, Qader, Hazem, Abdul}",505,95,1.0,"[95.0, 95.0, 95.0, 87.0]",94.75,"[100.0, 100.0, 100.0, 100.0]",match
2,6897,ISIL (Da'esh) and Al-Qaida,"{Saiyid, Am, Al, Abd, Man, Agha, Abdul, Manan}","{M, S, A}","[54, 58, 76, 83, 103, 136, 143, 154, 156, 157, 176, 508, 514, 515, 516, 517, 522, 524, 526, 528, 545, 548, 553, 556, 581, 593, 595, 599, 600, 603, 604, 641, 643, 644, 656, 659, 661, 676, 696, 727, 733, 739, 758, 760, 765, 779, 781, 796, 826, 840, 931, 965, 1064, 1065, 1069, 1092, 1102, 1924, 2193, 2208, 2700, 3144, 3225, 3341, 3361, 3663, 3741, 3793, 3862, 4142, 5268, 5271, 5279, 5294, 5416, 5417, 5499, 5616, 5619, 5623, 5793, 5804, 6084, 6095, 6101, 6113, 6114, 6116, 6130, 6133, 6206, 6211, 6223, 6228, 6230, 6231, 6238, 6303, 6305, 6309, ...]",610,"[Agha, Man, Al, Am, Abdul, Saiyid]","{Saiyid, Ag, Am, Al, Abd, Lmnn, Man, Agha, Abdul, Manan, Bd}",514,94,1.0,"[95.0, 87.0, 87.0, 87.0, 95.0, 100.0]",93.88,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0]",match
3,6899,ISIL (Da'esh) and Al-Qaida,"{Shahata, Thirwat, Tharwat, Abdallah, Shihata, Ali, Tarwat, Salah}","{A, S, T}","[13, 20, 23, 54, 58, 67, 76, 83, 87, 98, 157, 507, 515, 516, 553, 604, 727, 733, 758, 765, 779, 786, 789, 796, 826, 829, 840, 1880, 1883, 1886, 1888, 1892, 1896, 1924, 2193, 2208, 2921, 3080, 3085, 3225, 3341, 3862, 5279, 5357, 5610, 5793, 6101, 6113, 6114, 6116, 6231, 6306, 6496, 6584, 6619, 6625, 6695, 6696, 6916, 6917, 6973, 6982, 7027, 7069, 7077, 7094, 7137, 7166, 7294, 7300, 7336, 7343, 7359, 7361, 7406, 7434, 7483, 7492, 7496, 7504, 7524, 7556, 105424, 106138, 106544, 106548, 107009, 110103, 110164, 112198, 113224, 113334, 113787, 113926, 117506, 118875, 119026, 119200, 119561, 119633, ...]",339,"[Tarwat, Shahata, Abdallah, Ali, Salah]","{Shahata, Thirwat, Tharwat, Abdallah, Shihata, Ali, Tarwat, Salah}",796,97,1.0,"[100.0, 100.0, 100.0, 87.0, 95.0]",97.30,"[100.0, 100.0, 100.0, 100.0, 100.0]",match
4,6901,ISIL (Da'esh) and Al-Qaida,"{Chaudhry, Majid, Abdul, Majeed}","{C, M, A}","[83, 143, 154, 515, 517, 522, 526, 545, 548, 581, 603, 641, 643, 659, 661, 676, 696, 727, 1064, 1086, 1092, 1102, 3185, 3190, 3793, 5271, 5416, 5420, 5499, 5522, 5553, 5555, 5623, 6095, 6130, 6133, 6206, 6211, 6230, 6238, 6303, 6309, 6485, 6506, 6569, 6615, 6616, 6617, 6652, 6695, 6830, 6831, 6908, 6944, 6972, 6974, 6982, 7035, 7069, 7166, 7168, 7206, 7237, 7285, 7307, 7367, 7388, 7446, 7456, 7472, 7483, 7496, 7499, 7508, 105631, 10655

# 3. Summary Statistics

## 3.1 Results

In [7]:
#standard matching label summary
counts_standard=df_standard['Label'].value_counts().sort_index()
proportions_standard=df_standard['Label'].value_counts(normalize=True).sort_index()
summary_standard=pd.DataFrame({'Count': counts_standard, 'Proportion': proportions_standard})

#grouped label summary
counts_grouped=df_grouped['Label'].value_counts().sort_index()
proportions_grouped=df_grouped['Label'].value_counts(normalize=True).sort_index()
summary_grouped=pd.DataFrame({'Count': counts_grouped, 'Proportion': proportions_grouped})

#print results
print("Standard Matching Label Summary:\n", summary_standard)
print("\nGrouped Matching Label Summary:\n", summary_grouped)

Standard Matching Label Summary:
                    Count  Proportion
Label                               
match               2699    0.748060
not match            271    0.075111
preliminary match    638    0.176829

Grouped Matching Label Summary:
                    Count  Proportion
Label                               
match               2389    0.662140
not match            438    0.121397
preliminary match    781    0.216463


## 3.2 Discussion

- The `standard method`is more decisive, assigning 74.8% of cases as matches and only 7.5% as non-matches, with 17.7% marked as preliminary matches. This suggests it tends to commit more often, possibly at the cost of precision in ambiguous cases.

- In contrast, the `grouped method` is more cautious, labeling only 66.2% as matches and increasing both non-matches (12.2%) and especially preliminary matches (21.6%), showing a preference for deferring decisions when confidence is lower.

# 4. Labeling Validation

## 4.1 Methodology

To assess the performance of each name-matching method in attributing labels, we manually evaluated a sample of name pairs. The goal was to determine how accurately each method classified matches, near matches, and non-matches. For each method, the UK IDs of incorrectly labeled pairs were recorded and used to generate confusion matrices and calculate precision metrics, enabling a direct comparison between the two approaches.

#### 4.1.1 *Sample*

To a accomplish this, a random sample of 250 UK IDs was selected and used for both matching methods to ensure a fair comparison. Although the ideal sample size for a ±5% margin of error at 95% confidence is 385, a smaller sample of 250 offered a practical balance between statistical rigor and manual labeling effort.

#### 4.1.2  *Evaluation Metrics*

As outlined in previous notebooks, we adopted a three-tier labeling strategy (Match, Not Match, and Preliminary Match)  designed to maximize confidence in automatic decisions while reserving uncertain cases for human review. This strategy was developed to reduce manual effort in the context of financial sanctions, where accuracy is critical and false positives carry significant consequences.

The most important evaluation metrics are:

- **Precision** in `match` and `not match`: If the system labels a pair as a match, it should truly be a match. Likewise, if it labels a pair as not match, it should be genuinely safe to ignore. These represent the most confident outcomes and must be highly reliable.

- **Recall** in `not match` (important but secondary): This metric indicates how many actual non-matches are confidently identified. Higher recall reduces the number of names falling into the Preliminary Match category, thereby lowering manual review workload. However, ensuring precision remains more critical.

We do not directly optimize for the `Preliminary Match` class and instead rely on it to catch edge cases and potential true matches that lack clear confidence.


We do not directly optimize for the `preliminary match` class. Instead, it serves as a catch-all for edge cases and uncertain true matches that fall below the confidence threshold. According to our strategy, it is perfectly acceptable for a real match to be classified as preliminary match, since these cases will still be manually reviewed and not missed.

#### 4.1.3 *Human Evaluation*

Determining whether two names correspond to the same person is a highly subjective task, complicated by different languages, language structures, spelling variations, and nuances that often require linguistic expertise to fully understand. To maintain consistency when visually evaluating matched names, the following guidelines were established:

- A pair is considered a match only if there are at least three common words; otherwise, it is classified as a preliminary match to reflect lower confidence.

- Since human evaluators may have varying familiarity with different languages, all words were weighted equally, with no distinction made between first names, middle names, or surnames. This approach was chosen to maintain consistency across unknown language structures.

- When only one word matches, the pair is usually classified as a non-match, except in cases where the matching word is a distinctive or unusually long name, in which case it may be considered a preliminary match.

## 4.2 Results

In [8]:
#create samples 
sample_size=250
sample_s=df_standard.sample(n=sample_size, random_state=5).sort_values(by='UK ID', ascending=True).reset_index(drop=True)
sample_g=df_grouped.sample(n=sample_size, random_state=5).sort_values(by='UK ID', ascending=True).reset_index(drop=True)

In [9]:
#inspect standard macthes
sample_s.head(250).sort_values(by='Score',ascending=False)

,UK ID,UK Sanction Programme,UK Name,EU Name Match,EU ID,Score,String Length,Label
0,6894,ISIL (Da'esh) and Al-Qaida,Abdul Rahman Mohamad Iqbal,Mohamad Iqbal Abdul Rahman,1004,100,26,match
1,6905,Afghanistan,Abdul Bari Akhund,Abdul Bari Akhund,556,100,17,match
2,6912,Afghanistan,Abdul Rauf Khadem,Abdul Rauf Khadem,719,100,17,match
3,6932,ISIL (Da'esh) and Al-Qaida,Abu Qatada Al Filistini,Abu Qatada Al Filistini,836,100,23,match
4,7024,ISIL (Da'esh) and Al-Qaida,Mohamed Ben Belgacem Ben Abdallah Al Aouadi,Mohamed Ben Belgacem Ben Abdallah Al Aouadi,927,100,43,match
5,7099,ISIL (Da'esh) and Al-Qaida,Bilal Bin Marwan,Bin Marwan Bilal,557,100,16,match
6,7278,Afghanistan,Matiullah,Matiullah,649,100,9,match
7,7296,Afghanistan,Akhtar Mohammad Maz Hari,Akhtar Mohammad Maz Hari,651,100,24,match
8,7380,Afghanistan,Abdul Manan Nayazi,Abdul Manan Nayazi,702,100,18,match
9,7442,Afghanistan,Abdul Wahed Shafiq,Abdul Wahed Shafiq,758,100,18,match


In [10]:
#store wrongly labeled UK IDs from the standard sample

change_to_match_s=[14535,16307,13838,14479,14918]	

change_to_not_match_s=[16072,12725,14807,15962,15440,12852,16068,15052,16014,15420,16309,15783,15651,16472,16782]		

change_to_preliminary_match_s=[7278,7558,8258,11778,11933,11934,12216,12418,12507,12469,12769,12769,13333,13413,13497,13505,16595,16542,15862,15626,16381,16367,14329]	

In [11]:
#inspect grouped macthes
sample_g[['UK ID', 'UK Name', 'Name Overlap','EU Name Match','Multi Score','Coverage Ratio','Raw Scores','Label']].sort_values(by='Multi Score', ascending=False).head(250)


,UK ID,UK Name,Name Overlap,EU Name Match,Multi Score,Coverage Ratio,Raw Scores,Label
18,8904,"{Sheiman, Vladimirovich, Sheyman, Ulazimiravich, Viktor, Uladzimiravich, Uladimiravich, Viktar}","[Viktor, Sheiman, Vladimirovich]","{Sheiman, Uladzimiravitj, Vladimirovitj, Sjejman, Vladimirovich, Sheyman, Viktor, Uladzimiravich, Viktar}",100,1.000000,"[100.0, 100.0, 100.0]",match
16,8258,"{Chadian, Shahbandar, Samira}","[Samira, Chadian, Shahbandar]","{Chadian, Shahbandar, Samira}",100,1.000000,"[100.0, 100.0, 100.0]",match
48,12923,"{Vladimir, Andriyovych, Andreevich, Volodymyr, Konstantinov}","[Vladimir, Andreevich, Andriyovych, Volodymyr, Konstantinov]","{Konstantinov, Andrejevitj, Volodomyr, Vladimir, Andriyovych, Andreevich, Volodymyr, Wladimir, Konstantynow, Andrijovytj, Konstantynov}",100,1.000000,"[100.0, 100.0, 100.0, 100.0, 100.0]",match
56,13041,"{Degtyaryov, Degtyarev, Vladimirovich, Mikhail}","[Degtyaryov, Mikhail, Vladimirovich]","{Mikhail, Michail, Vladimirovitj, Vladimirovich, Degtyarev, Degtiarjov, Degtyaryov}",100,1.000000,"[100.0, 100.0, 100.0]",match
57,13068,"{Gromov, Alexeyevich, Alexey}","[Gromov, Alexeyevich, Alexey]","{Gromov, Alexey, Aleksei, Alexeyevich, Alekseevich}",100,1.000000,"[100.0, 100.0, 100.0]",match
51,12953,"{Menyailo, Sergei, Ivanovich}","[Ivanovich, Sergei, Menyailo]","{Ivanovich, Menjajlo, Iwanowitsch, Sergei, Menjailo, Sergej, Ivanovitj, Menyailo}",100,1.000000,"[100.0, 100.0, 100.0]",match
72,13394,"{Borisovna, Svetlana, Savchenko}","[Borisovna, Savchenko, Svetlana]","{Savtjenko, Borisovna, Svetlana, Savchenko}",100,1.000000,"[100.0, 100.0, 100.0]",match
64,13206,"{Kuzovlev, Tambov, Yurevich, Sergey, Kuzovliov, Ignatov}","[Yurevich, Ignatov, Kuzovlev, Tambov, Sergey]","{Jurjevitj, Kuzovlev, Tambov, Sergej, Yurevich, Sergey, Ignatov}",100,1.000000,"[100.0, 100.0, 100.0, 100.0, 100.0]",match
59,13073,"{Valerevich, Konstantin, Malofeev}","[Malofeev, Konstantin, Valerevich]","{Evich, Valerevich, Konstantin, Valer, Malofeev}",100,1.000000,"[100.0, 100.0, 100.0]",match
58,13071,"{Vadimovich, Serhiy, Sergiy, Vadymovych, Abisov, Sergey}","[Vadymovych, Serhiy, Abisov, Sergey]","{Vadimovich, Serhiy, Sergiy, Serhij, Vadymovych, Abisov, Vadimovitj, Vadymovytj, Sergej, Sergey}",100,1.000000,"[100.0, 100.0, 100.0, 100.0]",match


In [12]:
#store wrongly labeled UK IDs from the grouped sample

change_to_match_g=[]	

change_to_not_match_g=[15526,15651,14807]		

change_to_preliminary_match_g=[]	

In [13]:
#create a column with the true labels for each sample

sample_s['True Label']=sample_s['Label'].copy()
sample_g['True Label']=sample_g['Label'].copy()

sample_s.loc[sample_s['UK ID'].isin(change_to_match_s),'True Label']='match'
sample_s.loc[sample_s['UK ID'].isin(change_to_not_match_s),'True Label']='not match'
sample_s.loc[sample_s['UK ID'].isin(change_to_preliminary_match_s),'True Label']='preliminary match'

sample_g.loc[sample_s['UK ID'].isin(change_to_match_g),'True Label']='match'
sample_g.loc[sample_s['UK ID'].isin(change_to_not_match_g),'True Label']='not match'
sample_g.loc[sample_s['UK ID'].isin(change_to_preliminary_match_g),'True Label']='preliminary match'

In [14]:
class_labels=['not match','preliminary match', 'match']


#calculate classification report
report_s=classification_report(
    sample_s['True Label'], 
    sample_s['Label'], 
    labels=class_labels, 
    output_dict=True
)

#compute confusion matrix
confusion_matrix_s=confusion_matrix(sample_s['True Label'],sample_s['Label'], labels=class_labels)

#extract relevant metrics
precision_match_s=report_s['match']['precision']
precision_not_match_s=report_s['not match']['precision']
recall_not_match_s=report_s['not match']['recall']

#print results
print("Confusion Matrix:")
print(confusion_matrix_s)

print("\nClassification Report Summary:")
print(f"'Match' Precision:     {precision_match_s}")
print(f"'Not Match' Precision: {precision_not_match_s}")
print(f"'Not Match' Recall:    {recall_not_match_s}")

Confusion Matrix:
[[ 21  14   1]
 [  0  38  21]
 [  0   5 150]]

Classification Report Summary:
'Match' Precision:     0.872093023255814
'Not Match' Precision: 1.0
'Not Match' Recall:    0.5833333333333334


In [15]:
class_labels=['not match','preliminary match', 'match']


#calculate classification report
report_g=classification_report(
    sample_g['True Label'], 
    sample_g['Label'], 
    labels=class_labels, 
    output_dict=True
)

#compute confusion matrix
confusion_matrix_g=confusion_matrix(sample_g['True Label'],sample_g['Label'], labels=class_labels)

#extract relevant metrics
precision_match_g=report_g['match']['precision']
precision_not_match_g=report_g['not match']['precision']
recall_not_match_g=report_g['not match']['recall']

#print results
print("Confusion Matrix:")
print(confusion_matrix_g)

print("\nClassification Report Summary:")
print(f"'Match' Precision:     {precision_match_g}")
print(f"'Not Match' Precision: {precision_not_match_g}")
print(f"'Not Match' Recall:    {recall_not_match_g}")

Confusion Matrix:
[[ 39   3   0]
 [  0  52   0]
 [  0   0 156]]

Classification Report Summary:
'Match' Precision:     1.0
'Not Match' Precision: 1.0
'Not Match' Recall:    0.9285714285714286


## 4.3 Discussion

- The `grouped method` achieved perfect precision for both match and not match classes, and high not match recall (0.91), confirming it only commits to clear cases, consistent with its more conservative behavior observed earlier.

- The `standard method` showed slightly lower precision for match (0.87) and weak recall for not match (0.58), meaning it’s more willing to classify aggressively but at the cost of mislabeling.

This aligns with prior observations: the `standard method` is more decisive but riskier, while the `grouped method` is cautious and favors accuracy.

# 5. Disagreement Analysis 

In [16]:
standard_map=df_standard[['UK ID', 'UK Name', 'EU ID', 'EU Name Match', 'Label']].rename(
    columns={'EU ID': 'Standard Method', 'Label': 'Standard Label', 'EU Name Match': 'EU Name (Standard)','UK Name':'UK Name (Standard)'}
)

grouped_map=df_grouped[['UK ID','UK Name', 'EU ID', 'Name Overlap','EU Name Match', 'Label']].rename(
    columns={'EU ID': 'Grouped Method', 'Label': 'Grouped Label', 'EU Name Match': 'EU Name (Grouped)','UK Name':'UK Name (Grouped)'}
)

df_comparison=pd.merge(standard_map, grouped_map, on='UK ID', how='inner')

#ensure consistent types
df_comparison['Standard Method']=df_comparison['Standard Method'].astype('Int64')
df_comparison['Grouped Method']=df_comparison['Grouped Method'].astype('Int64')

def compare_ids(row):
    val1=row['Standard Method']
    val2=row['Grouped Method']
    if pd.isna(val1) or pd.isna(val2):
        return False
    return val1==val2

df_comparison['Same EU ID'] = df_comparison.apply(compare_ids, axis=1)

#filter out cases where the names were matched to the same EU ID + cases were a match was not found
df_mismatch=df_comparison[df_comparison['Same EU ID']==False]
df_mismatch=df_mismatch[~((df_mismatch['Standard Label']=='not match') & 
                            (df_mismatch['Grouped Label']=='not match'))]

#reorder columns for clarity
df_mismatch=df_mismatch[[
    'UK ID', 'UK Name (Standard)',
    'Standard Method', 'EU Name (Standard)', 'Standard Label',
    'UK Name (Grouped)','Grouped Method', 'Name Overlap','EU Name (Grouped)', 'Grouped Label'
]]

#print results
print(f"Total meaningful mismatches in EU IDs: {len(df_mismatch)}")
df_mismatch.head(100)

Total meaningful mismatches in EU IDs: 588


,UK ID,UK Name (Standard),Standard Method,EU Name (Standard),Standard Label,UK Name (Grouped),Grouped Method,Name Overlap,EU Name (Grouped),Grouped Label
37,6994,Khalid Sheikh Mohammed,191,Khalid Sheikh Mohammed,match,"{Khalid, Mohammed, Sheikh}",151409,"[Sheikh, Khalid, Muhammad]","{Muhammad, Khalid, Hanafi, Sejk, Schejk, Sheikh, Chalil}",match
43,7016,Abu Mohammed,4141,Abu Muhammad,preliminary match,"{Mohammed, Deen, Robi, Eddaouahiri, Ahmed, Fuad, Zawahry, Abdul, Mohamed, Nur, Zawahiri, Qader, Aziz, Moez, Dhawahri, Rabie, Abdel, Abu, Zawahari, Muhammed, Fatma, Al, Muaz, Rabi, Salim, Aiman, Ayman, Ahmad, Doctor}",3563,"[Muhammed, Abdul, Raji]","{F, Zoghbi, Ben, Abdul, Raji, Mohamed, Zgby, Fattah, Besir, Il, Mry, Farag, Khalil, Lazrag, Albdelfattah, Abdefattah, El, Libico, Muhammed, Khlyl, Zoghbai, Merai, Larzg, Meri, Lebachir, Faraj, Bdfth, Ila, Fredj, Zgbye, Singapore, Di}",preliminary match
105,7354,Mohammad Naim,654,Nik Mohammad,preliminary match,"{Mohammed, Mullah, Barech, Baric, Gul, Naimullah, Baraich, Kamran, Barich, Ghul, Mohammad, Bareh, Khudaidad, Zrae, Naim, Berich, Spen, Akhund, Naeem}",556,"[Mullah, Akhund, Bari]","{Mullah, Akhund, Zakir, Haji, Sahib, Abdul, Bari}",preliminary match
196,7860,Ali Barkani,6827,Ali Barakat,preliminary match,"{Mustafa, Fjamel, Moustfa, Mostefa, Djamal, Ali, Djamel, Mostafa, Kalad, Belkasam, Damel, Balkasam, Barkani, Bekasam}",1095,"[Ali, Belkassem]","{Ldyn, Nwr, Al, Faycal, Blqsm, Drissi, Belkassem, Noureddine, Bn, Ali, Abou, Ben, Ldrysy, Ly}",not match
207,7886,Usama Hamdan,152920,Usama Ramadan,preliminary match,"{Usama, Hamdan}",<NA>,None,None,not match
209,7888,Abu Umar,7482,Abu Umar,match,"{Mohammed, Abu, Musa, Mousa, Id, Dr, Umar, Marzook, Abou, Marzuq, Marzuk, Mohamed, Sa, Marzouk}",836,"[Umar, Abu, Mohammed]","{Samman, Mahmoud, Mohammed, Al, Qatada, Abu, Othman, Uthman, Umar, Ismail, Omar, Takfiri, Umr, Filistini}",preliminary match
218,8248,Ali Barzan Ibrahim Hasan Al Tikriti,1898,Ali Barzan Ibrahim Hasan Al Tikriti,match,"{Al, Ali, Tikriti, Hasan, Ibrahim, Barzan}",101,"[Barzan, Tikriti, Al, Ibrahim, Hassan]","{Al, Hassan, Tikriti, Ibrahim, Barzan}",match
254,8709,Jules Mutebutsi,3189,Jules Mutebutsi,match,"{Mutebutsi, Jules, Mutebuzi, Mutebusi}",168504,[Jules],"{Habyarimana, Mbitse, Mulumba, Mbitshemunda, Jean, Jules, Claude}",not match
262,8720,Abu Akram,5618,Abu Akram,match,"{Sabai, Karim, Youseff, I, Yusif, Hany, Elsayed, Sebai, Sababt, Sabaay, Sayyed, Tusnin, Elsebai, Sayyid, Abu, Hani, El, Yousef, Youssef, Al, Yusef, Akram, Siba}",6618,"[Al, Abu, Yusef, I, Haji]","{Abdol, I, Yusif, Abdul, Yusuf, Hajji, Reza, Abd, Shala, Hajj, Abdolreza, Abdorreza, Abu, Shahlai, Yasir, Haji, Haj, Shalai, Karkh, Al, Yusef, Shahla, Shahlaee}",preliminary match
300,10637,Ali Hoseynitash,5270,Ali Hoseynitash,match,"{Tash, Hoseini, Ali, Seyyed, Hosseini, Hoseynitash}",154125,"[Hosseini, Seyyed]","{Abbas, Seyyed, Hosseini}",preliminary match


In [17]:
standard_wins=[8248,8709,11901,11912,12176,12199,12211,12213,12729,12730,12781,13088] #12
grouped_wins=[7354,7860,7888,8720,11635,11647,11800,12221,12651,12725,12765,12872,2971,13084,13301,13369] #16
draw=[6994,7016,7886,10637,10643,10647,10834,10929,11030,11235,11644,11646,11708,11933,12013,12015,12020,12045,12053,12230,12240,12415,12481,
     12510,12650,12852,12894,12995,13086,13233,13252,13347] #32

It is worth further inspecting the cases in which the standard approach clearly picked a better match than the grouped approach for the UK ID to try to comprehend the limitations of the algorithm created to match the names. For thsi, we printed thsoe UK IDs fro mismatch and also the df_grouped for those UK IDs to see if there is some relationship between the metrcis and those errors. 

In [18]:
df_mismatch[df_mismatch['UK ID'].isin(standard_wins)]

,UK ID,UK Name (Standard),Standard Method,EU Name (Standard),Standard Label,UK Name (Grouped),Grouped Method,Name Overlap,EU Name (Grouped),Grouped Label
218,8248,Ali Barzan Ibrahim Hasan Al Tikriti,1898,Ali Barzan Ibrahim Hasan Al Tikriti,match,"{Al, Ali, Tikriti, Hasan, Ibrahim, Barzan}",101,"[Barzan, Tikriti, Al, Ibrahim, Hassan]","{Al, Hassan, Tikriti, Ibrahim, Barzan}",match
254,8709,Jules Mutebutsi,3189,Jules Mutebutsi,match,"{Mutebutsi, Jules, Mutebuzi, Mutebusi}",168504,[Jules],"{Habyarimana, Mbitse, Mulumba, Mbitshemunda, Jean, Jules, Claude}",not match
433,11901,Ali Mamlouk,6304,Ali Mamlouk,match,"{Mamlouk, Ali, Mamluk}",508,[Alfi],"{Al, Masri, Alfi, Ahmed, Abu, Abdullah, Saleh, Mariam, El, Mohamed}",not match
442,11912,Amjad Al Abbas,6309,Amjad Al Abbas,match,"{Abbas, Amjad, Al}",7359,"[Abbas, Al]","{Abbas, Al, Suleiman}",preliminary match
486,12176,Hassan Akharian,6583,Hassan Akharian,match,"{Akharian, Hossein, Hassan}",1892,"[Hassan, Hussein]","{Al, Hassan, Saddam, Ali, Tikriti, Hussein}",preliminary match
506,12199,Mohammad Bagher Bagheri,6606,Mohammad Bagher Bagheri,match,"{Bagheri, Bagher, Mohammad}",3808,"[Mohammad, Bagher]","{Zolqadr, Bagher, Mohammad, Zolghadr, Baqer, Zulqader}",preliminary match
517,12211,Ali Abdullah Ayyoub,6623,Ali Abdullah Ayyoub,match,"{Ayub, Abdallah, Abdullah, Ali, Ayoub, Ayyub, Ayyoub, Ayob}",154,[Abdallah],"{Abdallah, Adil, Mahdi}",not match
519,12213,Aous Aslan,6625,Aous Ali Aslan,preliminary match,"{Aslan, Aous}",145712,[Absalan],"{Absalan, Parviz}",not match
643,12729,Issam Hallaq,7092,Issam Hallaq,match,"{Essam, Issam, Hallak, Halaq, Barber, Hallaq}",146470,[Isa],"{Isa, Bayat}",not match
644,12730,Ezzedine Ismail,7093,Ezzedine Ismail,match,"{Ismail, Ismael, Ezzedine, Hamra}",6236,[Esmaili],"{Gholam, Hossein, Esmaili}",not match


In [19]:
df_grouped_errors=df_grouped[df_grouped['UK ID'].isin(standard_wins)]
df_grouped_errors=df_grouped_errors.sort_values('UK ID')
df_grouped_errors.head(30)

,UK ID,UK Sanction Programme,UK Name,UK Letters,Candidate EU IDs,Candidate Count,Name Overlap,EU Name Match,EU ID,Multi Score,Coverage Ratio,Length Adjusted Scores,Weighted Score,Raw Scores,Label
218,8248,Iraq,"{Al, Ali, Tikriti, Hasan, Ibrahim, Barzan}","{B, A, T, H, I}","[13, 25, 33, 39, 54, 65, 69, 87, 89, 98, 99, 101, 506, 544, 706, 829, 943, 945, 1016, 1888, 1892, 1894, 1896, 1898, 1900, 1920, 1922, 2325, 3080, 3081, 3082, 3084, 3085, 3225, 3340, 4140, 6047, 6049, 6060, 6063, 6074, 6078, 6079, 6080, 6082, 6177, 6457, 6480, 6612, 6825, 6889, 7150, 7294, 7370, 7477, 7478, 7498, 7601, 7602, 105748, 106534, 106538, 107679, 107693, 109872, 110845, 111553, 112075, 115225, 118547, 121825, 125574, 126488, 126714, 127732, 130197, 131720, 133626, 134079, 134380, 135748, 135879, 136340, 138684, 142836, 142906, 146768, 149736, 152809, 152900, 154528, 155925, 156392, 161670, 162355, 166170, 166475, 169200, 172452, 172959]",100,"[Barzan, Tikriti, Al, Ibrahim, Hassan]","{Al, Hassan, Tikriti, Ibrahim, Barzan}",101,97,1.000000,"[100.0, 100.0, 87.0, 100.0, 90.9090909090909]",96.69,"[100.0, 100.0, 100.0, 100.0, 90.9090909090909]",match
254,8709,Democratic Republic of the Congo,"{Mutebutsi, Jules, Mutebuzi, Mutebusi}","{M, J}","[614, 623, 630, 649, 690, 851, 1923, 4141, 5271, 6108, 6211, 6508, 6592, 6593, 6597, 6648, 6656, 6822, 6830, 6871, 7141, 7201, 7206, 7495, 7500, 105596, 106457, 107816, 108458, 109183, 109876, 110276, 115228, 115965, 117521, 117533, 123624, 124315, 124573, 124987, 125486, 127549, 131237, 134007, 134115, 134292, 134388, 134832, 135640, 138155, 138489, 138504, 138841, 140489, 145794, 146410, 148363, 149393, 149904, 149914, 153611, 155933, 156306, 159833, 161870, 164367, 164376, 167488, 168504, 172394]",70,[Jules],"{Habyarimana, Mbitse, Mulumba, Mbitshemunda, Jean, Jules, Claude}",168504,50,0.500000,[95.0],83.75,[100.0],not match
433,11901,Syria,"{Mamlouk, Ali, Mamluk}","{M, A}","[69, 83, 89, 103, 136, 139, 143, 154, 175, 508, 515, 517, 522, 523, 524, 526, 528, 540, 544, 545, 548, 552, 581, 582, 603, 614, 623, 641, 643, 647, 649, 651, 653, 659, 661, 669, 676, 696, 702, 717, 719, 727, 851, 945, 1064, 1065, 1092, 1102, 1923, 2194, 2700, 3000, 3144, 3782, 3793, 4141, 5271, 5416, 5417, 5499, 5623, 5804, 6079, 6095, 6096, 6108, 6130, 6133, 6206, 6211, 6230, 6238, 6303, 6305, 6309, 6485, 6506, 6555, 6569, 6597, 6605, 6614, 6615, 6616, 6617, 6630, 6656, 6695, 6822, 6830, 6867, 6888, 6908, 6944, 6972, 6974, 6982, 7035, 7069, 7072, ...]",362,[Alfi],"{Al, Masri, Alfi, Ahmed, Abu, Abdullah, Saleh, Mariam, El, Mohamed}",508,50,0.500000,[81.42857142857143],73.57,[85.71428571428572],not match
442,11912,Syria,"{Abbas, Amjad, Al}",{A},"[154, 513, 517, 523, 526, 552, 620, 733, 765, 1037, 1064, 1092, 3782, 3788, 5270, 5279, 5280, 5754, 5793, 5794, 6101, 6207, 6208, 6209, 6212, 6215, 6216, 6224, 6229, 6230, 6231, 6234, 6400, 6485, 6569, 6582, 6583, 6615, 6617, 6627, 6650, 6694, 6894, 6944, 6972, 6973, 6988, 7027, 7029, 7146, 7221, 7225, 7229, 7291, 7299, 7300, 7359, 7386, 7431, 7435, 7447, 7461, 7499, 7508, 7510, 7565, 7582, 7583, 105305, 106554, 110161, 113244, 113326, 115829, 117360, 117498, 118917, 119190, 119656, 119660, 121034, 122062, 125453, 125474, 125819, 126263, 126338, 126646, 126806, 126812, 127417, 129843, 130075, 130246, 130362, 132627, 133882, 134031, 134135, 134159, ...]",282,"[Abbas, Al]","{Abbas, Al, Suleiman}",7359,85,0.666667,"[95.0, 87.0]",84.92,"[100.0, 100.0]",preliminary match
486,12176,Iran,"{Akharian, Hossein, Hassan}","{H, A}","[13, 39, 65, 69, 89, 103, 175, 505, 521, 523, 528, 533, 542, 544, 552, 580, 590, 591, 603, 607, 651, 661, 669, 706, 829, 842, 944, 1065, 1888, 1892, 2193, 3782, 4140, 5270, 5352, 5416, 5803, 6047, 6074, 6078, 6079, 6080, 6082, 6130, 6207, 6224, 6240, 6312, 6457, 6495, 6503, 6555, 6583, 6605, 6614, 6616, 6621, 6694, 6887, 6896, 6898, 6960, 7073, 7075, 7080, 7085, 7087, 7132, 7134, 7144, 7146, 7150, 7205, 7262, 7362, 7368, 7386, 7447, 7481, 7498, 7501, 7513, 7540, 7586, 105305, 105748, 10613

## 5.1 Discussion

Out of the 3,608 unique UK individuals, the standard and grouped methods matched to different EU IDs in 587 cases. This already excludes instances where both methods selected different matches but classified them as not match.

From a manual review of the first 60 disagreement cases:

- 32/60 involved different but equally reasonable matches.

- 16/60 were cases where the grouped method found a significantly better match than the standard method — as expected.

- 12/60 showed the standard method produced a clearly better match, which was surprising given its simplicity as a baseline method.

Upon further investigation, those 12 unexpected failures revealed several limitations and blind spots in the grouped algorithm, including:

1. **Missing matches in candidate pool**: In some cases, the correct EU ID selected by the standard method wasn’t even included in the grouped method’s candidate list. This likely happened because the similarity threshold in the grouped method was too strict, causing it to overlook valid matches that scored just below the cutoff.
    
    - Example (UK ID 12729): `Issam Hallaq` was correctly matched by the standard method to EU ID 7092 (`Issam Hallaq`). However, the grouped method matched it to “Isa Bayat`, a completely different person. EU ID 7092 wasn’t even part of the candidate list, indicating the threshold filtered it out.
<br>

2. **Confusion within same family**: The grouped method sometimes confused individuals from the same family or name group. This was often due to short first names being ignored or underweighted—yet those are sometimes the only features that differentiate otherwise similar names.

   - Example (UK ID 8248): `Ali Barzan Ibrahim Hasan Al Tikriti` was correctly matched by the standard method. The grouped method picked another `Al Tikriti` family member, likely because it did not give enough importance to the short but distinctive token `Ali`.
<br>

3. **Over-simplification of similar words**: At times, the grouped method treated distinct names as if they were the same, especially when spellings were similar. This led to mismatches between people whose names look alike but are not actually the same.

   - Example (UK ID 12199): `Mohammad Bagher Bagheri` was matched correctly by the standard method. The grouped method, however, selected `Mohammad Bagher` instead, likely treating `Bagher` and `Bagheri` as mere variants of the same name, rather than distinct individuals.
<br>

4. **Too much weight on common words**: The algorithm occasionally gave excessive weight to generic name elements such as “`Mohammad`, `Al`, `Abu` etc. These frequent terms added noise to the comparison and sometimes led the model to overlook more distinctive name components.



# 6. Conclusion

The grouped method demonstrates clear superiority in accurately labeling matches, prioritizing precision and minimizing false positives. This cautious approach inevitably increases the number of cases requiring human review, which is expected given the inherent complexity of name matching across diverse languages and variations.

When it comes to selecting the best match, the two methods are broadly comparable. They disagreed in about 16% of cases, but roughly half of those disagreements involved equally valid alternatives. In the remaining cases, each method outperformed the other on some occasions, reflecting different strengths and weaknesses.

Notably, the few instances where the standard method outperformed the grouped method revealed blind spots in the grouped algorithm—limitations that stem from its design and thresholding, which sometimes cause it to miss the best matches.

Overall, while the grouped method excels in label accuracy and reducing false matches by capturing nuanced name information, fundamental algorithmic improvements are essential to ensure consistent performance. Without these, the method performs well most of the time but can fail drastically in certain cases.

# 7. Future Improvements

- Explore lowering the similarity threshold used in pre-selecting EU candidates to reduce the chance of missing good matches, while keeping computation time reasonable.

# 8. Key Limitations

1. **Subjectivity of the task:** Despite efforts to maintain consistency and minimize bias, evaluating name matches remains inherently subjective due to linguistic nuances and variations. However, in real-world sanctions screening, name matching is only one part of a broader verification process that includes multiple personal details, which helps mitigate this subjectivity.

3. **Sample Size Constraints:** The full dataset included 3,608 individuals, but manual evaluations were limited to smaller samples of 250 and 60 pairs due to resource constraints. These small sample sizes reduce the statistical robustness of the validation and disagreement analyses.

# 9. Output

Since a validated sample of the grouped matching results has already been created, we can now use it for further analysis as it has multiple metrics that can be explored. The final datasets were saved as follows:

In [20]:
sample_g.head()

,UK ID,UK Sanction Programme,UK Name,UK Letters,Candidate EU IDs,Candidate Count,Name Overlap,EU Name Match,EU ID,Multi Score,Coverage Ratio,Length Adjusted Scores,Weighted Score,Raw Scores,Label,True Label
0,6894,ISIL (Da'esh) and Al-Qaida,"{Fihiruddin, Abu, Muqti, A, Mohamad, Rahman, Jibril, Abdul, Abdurrahman, Iqbal, Fikiruddin}","{F, A, R, J, I, M}","[630, 643, 1004, 3140, 4686, 5240, 5262, 5271, 5623, 6133, 6211, 6478, 6494, 6830, 6974, 7250, 113355, 115714, 117974, 123615, 125562, 126101, 127538, 129864, 130225, 133935, 134828, 135060, 136530, 136975, 138176, 145803, 146560, 147032, 150914, 159126, 162479, 162975, 165652, 166799, 167049, 167477, 171171, 172374, 172394]",45,"[Jibril, Abdurrahman, Abu, A, Rahman, Fihiruddin, Mohamad, Iqbal, Muqti]","{Fihiruddin, Abu, Muqti, A, Mohamad, Rahman, Jibril, Abdul, Abdurrahman, Iqbal, Fikiruddin}",1004,97,1.0,"[100.0, 100, 87.0, 87.0, 100.0, 100, 100.0, 95.0, 95.0]",97.00,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]",match,match
1,6905,Afghanistan,"{Mullah, Akhund, Zakir, Haji, Sahib, Abdul, Bari}","{B, A, Z, H, M, S}","[25, 46, 54, 103, 514, 515, 528, 553, 556, 593, 595, 599, 600, 603, 604, 661, 727, 750, 927, 928, 937, 965, 966, 1016, 1065, 1069, 1085, 1102, 1894, 1896, 2193, 2208, 3084, 3144, 3225, 3341, 3361, 3663, 3808, 4142, 5294, 5295, 5416, 5417, 5529, 5616, 5619, 5625, 6044, 6047, 6049, 6052, 6053, 6054, 6073, 6074, 6078, 6079, 6084, 6085, 6090, 6116, 6130, 6213, 6223, 6238, 6309, 6484, 6494, 6607, 6608, 6616, 6695, 6898, 6947, 6982, 7065, 7069, 7074, 7166, 7199, 7223, 7231, 7443, 7472, 7474, 7475, 7477, 7483, 7484, 7496, 7597, 106138, 106144, 106385, 106389, 106534, 106538, 106584, 106654, ...]",196,"[Sahib, Bari, Akhund, Zakir, Mullah, Abdul, Haji]","{Mullah, Akhund, Zakir, Haji, Sahib, Abdul, Bari}",556,97,1.0,"[95.0, 95.0, 100.0, 95.0, 100.0, 95.0, 95.0]",97.32,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]",match,match
2,6912,Afghanistan,"{Khadem, Aliza, Abdul, Rauf}","{K, A, R}","[143, 146, 513, 659, 676, 719, 724, 786, 849, 1096, 2921, 3062, 3140, 3342, 3793, 5266, 5925, 6212, 6215, 6216, 6240, 6582, 6584, 6650, 6945, 6974, 6989, 6993, 7085, 7221, 7229, 7237, 7250, 7291, 7361, 7368, 7405, 7435, 7455, 7456, 7504, 7513, 7519, 7523, 7567, 107818, 108640, 109244, 109980, 110174, 110425, 113326, 117485, 117974, 119548, 119656, 123615, 125538, 125637, 126338, 126507, 126698, 127526, 127528, 127726, 128270, 128279, 129815, 129836, 129884, 130225, 130355, 132522, 133907, 133959, 134035, 134083, 134107, 134171, 134175, 134340, 134484, 134488, 134532, 134556, 134684, 134860, 134872, 134876, 134880, 134884, 134892, 134900, 134920, 134924, 134928, 134932, 134936, 134964, 134976, ...]",248,"[Abdul, Aliza, Khadem, Rauf]","{Khadem, Mullah, Aliza, Abdul, Rauf}",719,97,1.0,"[95.0, 95.0, 100.0, 95.0]",97.19,"[100.0, 100.0, 100.0, 100.0]",match,match
3,6932,ISIL (Da'esh) and Al-Qaida,"{Samman, Mahmoud, Mohammed, Al, Qatada, Abu, Othman, Uthman, Umar, Ismail, Omar, Takfiri, Umr, Filistini}","{F, T, A, Q, O, I, M, U, S}","[23, 58, 157, 550, 836, 3081, 3082, 3083, 3225, 3962, 5609, 6515, 7025, 7478, 7484, 7497, 115714, 117251, 139991, 152885]",20,"[Umar, Takfiri, Filistini, Othman, Qatada, Ismail, Abu, Al, Samman, Mahmoud, Mohammed]","{Samman, Mahmoud, Mohammed, Al, Qatada, Abu, Othman, Uthman, Umar, Ismail, Omar, Takfiri, Umr, Filistini}",836,98,1.0,"[95.0, 100.0, 100, 100.0, 100.0, 100.0, 87.0, 87.0, 100.0, 100.0, 100.0]",97.89,"[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]",match,match
4,7024,ISIL (Da'esh) and Al-Qaida,"{Aouadi, Belgacem, Al, Abdallah, Ben, Hannachi, Fathi, Mohamed, Belkacem}","{B, F, A, H, M}","[25, 54, 69, 89, 103, 139, 175, 191, 528, 544, 556, 603, 651, 653, 661, 669, 702, 706, 923, 927, 928, 1016, 1065, 1085, 1894, 2700, 3000, 3060, 3144, 3181, 4140, 5416, 5417, 5500, 5619, 5623, 5625, 6047, 6049, 6052, 6074, 6078, 6079, 6080, 6082, 6096, 6118, 6130, 6133, 6213, 6309, 6457, 6555, 6605, 6613, 6614, 6616, 688

In [22]:
df_grouped.to_pickle(processed_dir/'automation_data.pkl')
sample_g.to_pickle(processed_dir/'automation_sample_labeled.pkl')